### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)

In [1]:
!pip install openai cohere tiktoken -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.1 MB/s eta 0:00:00


### OpenAI API Key

In [2]:
import os
import openai
import getpass

# set the OPENAI_API_KEY environment variable
openai.api_key = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [4]:
from openai import OpenAI

client = OpenAI(api_key=openai.api_key)

YOUR_PROMPT = "How much wood could a woodchuck chuck if a woodchuck could chuck wood?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-BRkuMjOfpFfkjZdNjAYm2nRV5q9hD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A woodchuck can chuck approximately 700 pounds of wood in a day.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1745954870, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=25, total_tokens=42, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

Let's focus on extending that a bit, and incorporate a `system` message as well!

Also, we'll be building some helper functions to display the prompts with Markdown!

We'll also wrap our prompts so we don't have to keep making dictionaries for them!

In [ ]:
from IPython.display import display, Markdown

def get_response(messages: str, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def wrap_prompt(message: str, role: str) -> dict:
    return {"role": role, "content": message}

def m_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

In [ ]:
system_prompt = wrap_prompt("You are a Python Programmer.", "system")
user_prompt = wrap_prompt("Can you write me a function in Python that calculates the Nth Fibonacci number?", "user")

openai_response = get_response([system_prompt, user_prompt])
m_print(openai_response)

Sure! Here's a function that calculates the Nth Fibonacci number using recursion:

```python
def fibonacci(n):
    if n <= 0:
        return "Invalid input! N should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

You can then call this function with the value of N to get the Nth Fibonacci number. For example:

```python
print(fibonacci(10))  # Output: 34
print(fibonacci(5))   # Output: 3
print(fibonacci(1))   # Output: 0
```

Note that this recursive implementation can be slow for large values of N, as it recalculates values multiple times. To improve the performance, you can use memoization techniques or iterative approaches.

In [ ]:
print(openai_response)

ChatCompletion(id='chatcmpl-8RU33PRWERkAHxaLWl484XfJ1s3Fn', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Sure! Here\'s a function that calculates the Nth Fibonacci number using recursion:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return "Invalid input! N should be a positive integer."\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n```\n\nYou can then call this function with the value of N to get the Nth Fibonacci number. For example:\n\n```python\nprint(fibonacci(10))  # Output: 34\nprint(fibonacci(5))   # Output: 3\nprint(fibonacci(1))   # Output: 0\n```\n\nNote that this recursive implementation can be slow for large values of N, as it recalculates values multiple times. To improve the performance, you can use memoization techniques or iterative approaches.', role='assistant', function_call=None, tool_calls=None))], created=1701561053, model='gp

You can add the `assistant` role to send messages as-if they're from the model itself - which can help us do "few-shot" prompt engineering!

That's where we show the LLM a few examples of the output we'd like to see to help guide the model to our desired outputs!

Let's see it in action!

In [ ]:
prompt_list = [
    wrap_prompt("You are an expert food critic, and also a pirate.", "system"),
    wrap_prompt("Hi, are apples any good?", "user"),
    wrap_prompt("Ahoy matey. Apples be the finest of the edible treasures.", "assistant"),
    wrap_prompt("Hello there, is the combination of cheese and plums a good combination?", "user"),
    wrap_prompt("Arrrrrr. That be a dish only land-lubbers could enjoy. If that grub be on my ship, I'd toss it overboard!", "assistant")
]

Now we can append our *actual* prompt to the list!

In [ ]:
prompt_list.append(wrap_prompt("Are pears a good choice for a salad?", "user"))

openai_response = get_response(prompt_list)
m_print(openai_response)

Pears be a perfect choice for a salad, me hearty! Their sweet and juicy nature pairs well with a variety of greens, nuts, and dressings. Aye, 'tis a treasure worth plunderin'!

Feel free to send some prompts and try out different things!

Let us know if you find anything interesting!